In [1]:
# Importation des bibliothèques
import pandas as pd 
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from scipy.stats import uniform, randint
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score

In [2]:
# ====== IMPORTATION DES DONNEES ======

supply = pd.read_excel(r"D:\Projects\IT\Data Science & IA\Developpement_Tableau_de_Bord_de_la_Chaine_Approvisionnement_Power_BI\data\supply-chain.xlsx")
print("Jeu de données importé✅✅")

Jeu de données importé✅✅


In [4]:
supply

,Delivery Status,Stock Level,Sales,Transportation Cost,Restock Needed,Region,Delivery Urgency,Estimated_day
0,Late,55,45,644.92,Yes,East,On Time,17
1,On time,77,18,440.75,No,South,On Time,17
2,Late,97,16,310.18,No,West,Late,9
3,On time,136,25,213.36,No,North,On Time,17
4,On time,175,25,832.97,No,North,On Time,17
...,...,...,...,...,...,...,...,...
695,Missing,50,31,943.21,Yes,South,Late,13
696,On time,155,33,681.76,No,West,On Schedule,9
697,Late,63,23,735.38,No,West,On Time,17
698,Late,166,22,284.31,No,East,On Time,17


In [5]:
# ====== PRETRAITEMENT DES DONNEES ======

# Séparer des features et de la cible 
x = supply.drop(columns=["Delivery Status", "Restock Needed"]) #Features
y = supply["Delivery Status"] #Target

# Encoder la cible (Target = Delivery Status)
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Diviser les données en entraînement et en test
x_train, x_test,y_train, y_test = train_test_split(x,y, random_state=42, test_size=0.2)

# Détecter les types de données
num_col = supply.select_dtypes(include=["int64","float64"]).columns.tolist()
cat_col =supply.select_dtypes(include=["object"]).columns.difference(["Delivery Status","Restock Needed"]).tolist()

# Prétraitement des données
num_transformer = Pipeline([
    ('Imputer', SimpleImputer(strategy='constant', fill_value=-1)),
    ('scaler', MinMaxScaler())
])

cat_transformer = Pipeline([
    ('Imputer', SimpleImputer(strategy='constant', fill_value='Missing')),
    ('oneencoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

# ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num',num_transformer,num_col),
        ('cat',cat_transformer, cat_col)
    ]
)
print("Prétraitement terminée✅✅")

Prétraitement terminée✅✅


In [6]:
# ====== ENTRAINEMENT & PREDICTION ======

# Modèles à entraîner
models = {
    "logistic" : LogisticRegression(max_iter=1000, solver='liblinear', class_weight="balanced"),
    "random_forest" : RandomForestClassifier(),
    'xgboost': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
}

# Hyperparamètres pour ²RandomizedSearchCV
param_dist = {
    "logistic":{
        'classifier__C': [0.1,1,10],
        'classifier__penalty': ["l1","l2"]
    },
    "random_forest":{
        'classifier__n_estimators':[100,200,300],
        'classifier__max_depth':[3,6,10],
        'classifier__min_samples_split': [2,5,10]
    },
    "xgboost":{
        'classifier__learning_rate': [0.01,0.05,0.1,0.2],
        'classifier__n_estimators': [100,200,300],
        'classifier__max_depth':[3,6,10],
        'classifier__subsample': [0.7,0.8,0.9],
        'classifier__colsample_bytree':[0.7,0.8,1],
        'classifier__game':[0,0.1,0.2]
    }
}

# Entraînement avec RandomizedSearchCV
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
best_models = {}

for name, model in models.items():
    print(f"\n Entraînement du modèle : {name}")
    
    pipe = Pipeline([
        ('preprocessing', preprocessor),
        ('classifier', model)
    ])
    
    random_search = RandomizedSearchCV(
        pipe,
        param_distributions=param_dist[name],
        n_iter=20,
        cv=cv,
        n_jobs=-1,
        scoring='accuracy',
        random_state=42
    )
    
    random_search.fit(x_train, y_train)
    best_models[name] = random_search.best_estimator_
print("Entrainement mis en place ✅✅")


 Entraînement du modèle : logistic


c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 6 is smaller than n_iter=20. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(



 Entraînement du modèle : random_forest

 Entraînement du modèle : xgboost


c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:1108: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
  warnings.warn(
c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:38:51] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "game", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Entrainement mis en place ✅✅


In [7]:
# ====== FONCTION DE LOG DANS MLFLOW ======

def mlflow_log(model, x_test, y_test, model_name): 
    try : 
        # Prédiction et calcul des métriques
        y_pred = model.predict(x_test)
        acc = accuracy_score(y_pred, y_test)
        f1 = f1_score(y_pred, y_test, average="weighted")
        rec = recall_score(y_pred, y_test, average="weighted")
        
        with mlflow.start_run(run_name=model_name):
            # Log des param_tres et des métriques
            mlflow.log_param('model_types', model_name)
            mlflow.log_params(model.get_params())
            mlflow.log_metric('accuracy', acc)
            mlflow.log_metric('f1_score',f1)
            mlflow.log_metric('recall', rec)
            
            # Log du modèle dans Mlflow
            mlflow.sklearn.log_model(model, model_name)
            
            # Enrégistrement dans Model Registry
            model_uri = f"run:/{mlflow.active_run().info.run_id}/{model_name}"
            result = mlflow.register_model(model_uri = model_uri, name = model_name)
            print(f"Model {model_name} enregistré dans le Model Registry avec succès ✅✅")
            
            # Promotion automatique en Production
            client = MlflowClient()
            client.transition_model_version_stage(
                name = model_name,
                version = result.version,
                stage="Production",
                archive_existing_versions=True
            )
        print(f"✅ {model_name} enregistré avec succès dans MLflow\n")
    except Exception as e:
        print(f"❌ Une erreur est survenue lors du logging du modèle {model_name} : {e}")

In [8]:
# ====== ENREGISTREMENT DES MODELES DANS MLFLOW ======
for name, best_model in best_models.items():
    mlflow_log(best_model, x_test, y_test, name)

print("🚀 Tous les modèles ont été entraînée et enregistrés avec succès dans MLflow ✅✅")

2025/05/15 17:39:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'logistic'.
Created version '1' of model 'logistic'.
C:\Users\hp\AppData\Local\Temp\ipykernel_4344\2218991927.py:29: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


Model logistic enregistré dans le Model Registry avec succès ✅✅
✅ logistic enregistré avec succès dans MLflow



2025/05/15 17:39:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'random_forest'.


Model random_forest enregistré dans le Model Registry avec succès ✅✅
✅ random_forest enregistré avec succès dans MLflow

❌ Une erreur est survenue lors du logging du modèle xgboost : 'super' object has no attribute '__sklearn_tags__'
🚀 Tous les modèles ont été entraînée et enregistrés avec succès dans MLflow ✅✅


Created version '1' of model 'random_forest'.
C:\Users\hp\AppData\Local\Temp\ipykernel_4344\2218991927.py:29: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
